In [ ]:
!pip install swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import gymnasium as gym
from gymnasium.wrappers import FlattenObservation

from typing import Optional
import numpy as np
import wandb
from tqdm import trange
wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [ ]:
class WANDBVideo(gym.Wrapper):
    def __init__(
        self,
        env: gym.Env,
        name: str = "video",
        pixel_hw: int = 84,
        render_kwargs={},
        max_videos: Optional[int] = None,
    ):
        super().__init__(env)

        self._name = name
        self._pixel_hw = pixel_hw
        self._render_kwargs = render_kwargs
        self._max_videos = max_videos
        self._video = []

    def _add_frame(self, obs):
        if self._max_videos is not None and self._max_videos <= 0:
            return
        if isinstance(obs, dict) and "pixels" in obs:
            if obs["pixels"].ndim == 4:
                self._video.append(obs["pixels"][..., -1])
            else:
                self._video.append(obs["pixels"])
        else:
            self._video.append(
                self.render(
                    # height=self._pixel_hw,
                    # width=self._pixel_hw,
                    # mode="rgb_array",
                    # **self._render_kwargs
                )
            )

    def reset(self, **kwargs):
        self._video.clear()
        obs = super().reset(**kwargs)
        self._add_frame(obs)
        return obs

    def step(self, action: np.ndarray):

        obs, reward, terminated, truncated, info = super().step(action)
        done = terminated or truncated
        self._add_frame(obs)

        if done and len(self._video) > 0:
            if self._max_videos is not None:
                self._max_videos -= 1
            video = np.moveaxis(np.stack(self._video), -1, 1)
            if video.shape[1] == 1:
                video = np.repeat(video, 3, 1)
            video = wandb.Video(video, fps=20, format="mp4")
            wandb.log({self._name: video}, commit=False)

        return obs, reward, done, info


In [ ]:
class UniversalSeed(gym.Wrapper):
    def seed(self, seed: int):
        seeds = self.env.seed(seed)
        self.env.observation_space.seed(seed)
        self.env.action_space.seed(seed)
        return seeds

def wrap_gym(env: gym.Env, rescale_actions: bool = True) -> gym.Env:
    # env = SinglePrecision(env)
    env = UniversalSeed(env)
    if rescale_actions:
        env = gym.wrappers.RescaleAction(env, -1, 1)

    if isinstance(env.observation_space, gym.spaces.Dict):
        env = FlattenObservation(env)

    # env = gym.wrappers.ClipAction(env)

    return env

In [ ]:
env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
env = wrap_gym(env, rescale_actions=True)
env = WANDBVideo(env)
# env.seed(42)
observation, _ = env.reset()
done = True
env.action_space

Box(-1.0, 1.0, (4,), float32)

In [ ]:
for i in trange(1000):
    action = env.action_space.sample()
    next_observation, reward, done, info = env.step(action)
    if done:
        print('DONE!')
        break

  9%|▉         | 88/1000 [00:00<00:06, 137.77it/s]/usr/local/lib/python3.11/dist-packages/moviepy/config_defaults.py:1: DeprecationWarning: invalid escape sequence '\P'
  """
/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: DeprecationWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
  from scipy.ndimage.filters import sobel

  if event.key is 'enter':

  9%|▉         | 94/1000 [00:02<00:22, 39.48it/s] 


Error: You must call wandb.init() before wandb.log()

In [ ]:
observation.shape

(24,)

In [ ]:
next_observation

array([-0.02854241, -0.01509872, -0.01178587, -0.02922622,  0.4477863 ,
       -0.34660098,  0.1419816 ,  0.3074001 ,  1.        ,  0.45907825,
        0.99999994, -0.04240048, -0.99999994,  0.        ,  0.44475374,
        0.44980457,  0.4655467 ,  0.49392548,  0.5388763 ,  0.6078454 ,
        0.7154868 ,  0.89384973,  1.        ,  1.        ], dtype=float32)

In [ ]:
reward

-0.13556052140891553

In [ ]:
done

False